# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,whitehorse,60.7161,-135.0538,-12.57,85,100,1.03,CA,1704172856
1,1,puerto ayora,-0.7393,-90.3518,24.75,98,47,2.40,EC,1704172857
2,2,adamstown,-25.0660,-130.1015,23.52,84,100,3.68,PN,1704172858
3,3,faaa,-17.5500,-149.6000,27.98,78,75,3.09,PF,1704172859
4,4,sao felix do xingu,-6.6447,-51.9950,22.15,98,47,0.39,BR,1704172860


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
#Update Dataframe to use Title_case for the city in order to display it on the map in title case. 
city_data_df["City"] = city_data_df["City"].str.title()
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Whitehorse,60.7161,-135.0538,-12.57,85,100,1.03,CA,1704172856
1,1,Puerto Ayora,-0.7393,-90.3518,24.75,98,47,2.40,EC,1704172857
2,2,Adamstown,-25.0660,-130.1015,23.52,84,100,3.68,PN,1704172858
3,3,Faaa,-17.5500,-149.6000,27.98,78,75,3.09,PF,1704172859
4,4,Sao Felix Do Xingu,-6.6447,-51.9950,22.15,98,47,0.39,BR,1704172860


In [19]:
%%capture --no-display

# Configure the map plot
# Made Legend = False to hide the legend. Too many ciites and display partial set of them does not add value (city name available on hover)
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 450,
    color = "City",
    size = "Humidity",
    legend = False)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [20]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities = city_data_df.loc[(city_data_df["Max Temp"]> 20) & (city_data_df["Max Temp"] < 25) &                     
                (city_data_df["Cloudiness"] < 10) & (city_data_df["Humidity"] < 85),:]

# Drop any rows with null values
ideal_cities= ideal_cities.dropna(how= "any").reset_index(drop = True)

# Display sample data
ideal_cities.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,11,Naliya,23.2667,68.8333,23.52,23,0,5.55,IN,1704172867
1,37,Jaleshwar,26.6471,85.8008,21.14,41,0,1.69,NP,1704172895
2,57,Sukuta,13.4103,-16.7082,21.01,64,0,0.00,GM,1704172914
3,98,Tura,25.5198,90.2201,23.32,41,0,0.81,IN,1704172956
4,131,La'Ie,21.6477,-157.9253,23.98,82,0,1.79,US,1704172991


### Step 3: Create a new DataFrame called `hotel_df`.

In [21]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,Naliya,IN,23.2667,68.8333,23,
1,Jaleshwar,NP,26.6471,85.8008,41,
2,Sukuta,GM,13.4103,-16.7082,64,
3,Tura,IN,25.5198,90.2201,41,
4,La'Ie,US,21.6477,-157.9253,82,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 5    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Naliya - nearest hotel: No hotel found
Jaleshwar - nearest hotel: Bajeko Satuwa
Sukuta - nearest hotel: London Inn
Tura - nearest hotel: No hotel found
La'Ie - nearest hotel: No hotel found
Kerikeri - nearest hotel: Avalon Resort
Limon - nearest hotel: Hotel NG
San Luis De La Loma - nearest hotel: No hotel found
Copala - nearest hotel: No hotel found
Pitimbu - nearest hotel: Pousada Brisa Mar
Sur - nearest hotel: Sur Hotel
Sehwan - nearest hotel: Dum Dastagir Resthouse
Berbera - nearest hotel: Al Madina Hotel
Kasane - nearest hotel: Kasane Travel Lodge


,City,Country,Lat,Lng,Humidity,Hotel Name
0,Naliya,IN,23.2667,68.8333,23,No hotel found
1,Jaleshwar,NP,26.6471,85.8008,41,Bajeko Satuwa
2,Sukuta,GM,13.4103,-16.7082,64,London Inn
3,Tura,IN,25.5198,90.2201,41,No hotel found
4,La'Ie,US,21.6477,-157.9253,82,No hotel found
5,Kerikeri,NZ,-35.2268,173.9474,72,Avalon Resort
6,Limon,CR,10.0000,-83.0333,83,Hotel NG
7,San Luis De La Loma,MX,17.2706,-100.8939,66,No hotel found
8,Copala,MX,16.6500,-98.9833,61,No hotel found
9,Pitimbu,BR,-7.4706,-34.8086,83,Pousada Brisa Mar


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 450,
    color = "City",
    size = "Humidity",
    hover = True,
    hover_cols = ["Hotel Name","Country"]
    )

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)